In [ ]:
import cv2
import numpy as np
import os

# Define input and output directories
input_dir = "INPUT1"  # Update with your actual input directory path
output_dir = "HEALTH"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define result file path
result_file_path = os.path.join(output_dir, "Health_status.txt")

def calculate_plant_health(image_path):
    """Calculates the health score and status of the plant based on image analysis."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at {image_path}")
        return None

    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color ranges for health classification
    green_lower, green_upper = np.array([35, 40, 20]), np.array([85, 255, 255])  # Healthy (Green)
    yellow_lower, yellow_upper = np.array([20, 100, 100]), np.array([35, 255, 255])  # Deficiency (Yellow)
    brown_lower, brown_upper = np.array([5, 50, 50]), np.array([20, 255, 255])  # Diseased (Brown)

    # Create masks
    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    brown_mask = cv2.inRange(hsv, brown_lower, brown_upper)

    # Count pixels in each category
    total_pixels = image.shape[0] * image.shape[1]
    green_ratio = (cv2.countNonZero(green_mask) / total_pixels) * 100
    yellow_ratio = (cv2.countNonZero(yellow_mask) / total_pixels) * 100
    brown_ratio = (cv2.countNonZero(brown_mask) / total_pixels) * 100

    # Calculate health score
    health_score = round(100 - (yellow_ratio + brown_ratio), 2)  # Ensures only positive values

    # Determine health condition
    if health_score > 70:
        health_status = "Healthy 🌱"
    elif health_score > 50:
        health_status = "Moderate Health ⚠"
    else:
        health_status = "Plant is Diseased ❌"

    return health_status, health_score, green_mask, yellow_mask, brown_mask

# Open the result file with UTF-8 encoding
with open(result_file_path, "w", encoding="utf-8") as result_file:
    # Iterate over all image files in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):  # Supports multiple image formats
            image_path = os.path.join(input_dir, filename)

            # Process image
            result = calculate_plant_health(image_path)

            if result:
                health_status, health_score, green_mask, yellow_mask, brown_mask = result

                # Write health status to file with UTF-8 encoding
                result_file.write(f"{filename}: {health_status} (Score: {health_score}%)\n")

                # Save masked images to output directory
                cv2.imwrite(os.path.join(output_dir, f"green_mask_{filename}"), green_mask)
                cv2.imwrite(os.path.join(output_dir, f"yellow_mask_{filename}"), yellow_mask)
                cv2.imwrite(os.path.join(output_dir, f"brown_mask_{filename}"), brown_mask)

                print(f"Processed: {filename} - {health_status} (Score: {health_score}%)")

print("Processing complete. Results saved in", result_file_path)
cv2.destroyAllWindows()